In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
%matplotlib inline
import os
import torch


In [ ]:
!pip install /kaggle/input/aefficientnet/efficientnet_pytorch-0.7.0-py3-none-any.whl


In [ ]:
from efficientnet_pytorch import EfficientNet
model_transfer = EfficientNet.from_name('efficientnet-b3')

In [ ]:
import torch.nn as nn

In [ ]:
model_transfer._fc=nn.Linear(1536,5,bias=True)

In [ ]:
model_transfer.load_state_dict(torch.load('../input/aefficientnet/weights.pt'))

In [ ]:
import cv2

In [ ]:
test_path = '../input/cassava-leaf-disease-classification/test_images/'
sample = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

In [ ]:
import torch
from torch.utils.data import Dataset ,DataLoader
import pytorch_lightning as pl
from torch import nn
from torchvision import datasets
import torchvision.transforms as transforms
import cv2

In [ ]:
class LeafDataset(Dataset):
    
    def __init__(self, dataframe, transform=None, test=False):
        self.df = dataframe
        self.transform = transform
        self.test = test
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        
        label = self.df.label.values[idx]
        p = self.df.image_id.values[idx]
        
        if self.test == False:
            p_path = train_path + p
        else:
            p_path = test_path + p
            
        image = cv2.imread(p_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = transforms.ToPILImage()(image)
        
        if self.transform:
            image = self.transform(image)
        
        return image,torch.tensor(label)

In [ ]:

test_transforms = transforms.Compose([transforms.Resize(550),
                                       transforms.CenterCrop(512),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                                 [0.229, 0.224, 0.225])]
                                       )


In [ ]:
testset = LeafDataset(sample, transform=test_transforms,test=True)

In [ ]:
testLoader  = torch.utils.data.DataLoader(testset,
                                           batch_size=1,
                                           shuffle=False,
                                           num_workers=0)

In [ ]:
loaders = {'test' : testLoader}

In [ ]:
import torch.optim as optim
criterion= nn.CrossEntropyLoss()
optimizer= optim.Adam(model_transfer._fc.parameters(),lr=5e-4)
use_cuda=torch.cuda.is_available()

In [ ]:
model_transfer=model_transfer.cuda()

In [ ]:
test_pred = []

model_transfer.eval()
for batch_idx, (datatest, targettest) in enumerate(loaders['test']):
    if use_cuda:
        datatest, targettest = datatest.cuda(), targettest.cuda()
    pred= model_transfer(datatest)
    pred = pred.argmax(1).cpu().detach().numpy().astype('int')
    test_pred.extend(pred)

sample.label = test_pred
sample.to_csv('submission.csv',index=False)
   

In [ ]:
!cat submission.csv